## Imports

In [132]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re  # Import the regular expression module
from tqdm import tqdm

## Scraping

In [133]:
SCRAPE_SOURCE = "https://www.worldometers.info/demographics/"
SCRAPE_ROOT = "https://www.worldometers.info"
response = requests.get(SCRAPE_SOURCE)
response.raise_for_status()  # Raise an exception for bad status codes
soup = BeautifulSoup(response.content, 'html.parser')

In [138]:
# Initialize a list to collect demographic data
demographics_data = []

df_demographics = pd.DataFrame(columns=["Country",
                                        "LifeExpectancy_Both",  # (Both Sexes, in years)
                                        "LifeExpectancy_Female",  # (Females, in years)
                                        "LifeExpectancy_Male",  # (Males, in years)
                                        "UrbanPopulation_Percentage",  # (percentage without commas)
                                        "UrbanPopulation_Absolute",  # (if available)
                                        "PopulationDensity"])
progress_bar = tqdm(soup.find_all(attrs={'data-country': True}))
for country_link in progress_bar:
    progress_bar.set_description(f"Scraping data for {country_link.text.strip()}")

    country_url = SCRAPE_ROOT + soup.find_all(attrs={'data-country': True})[0]['href']
    country_url = SCRAPE_ROOT + country_link['href']  # Fixed to use current country_link

    response = requests.get(country_url)
    country_soup = BeautifulSoup(response.content, 'html.parser')

    # Get life expectancies
    expectancies = map( lambda x: float(x.text.strip()), country_soup.find_all(attrs={'class': 'grid grid-col-1 lg:grid-cols-3 gap-4'})[0].find_all(
                          attrs={'class': 'text-2xl font-bold mb-1.5'}))

    expectancies = list(expectancies)  # Convert map object to list

    # Get urban population data
    populations= re.findall('\d+\.?\d+',
                        country_soup.find(lambda tag: tag.name == 'p' and 'Currently' in tag.text).text.replace(',', ''))[0:2]
    # Dealing with lack of absolute number
    if len(populations) == 1:
        populations.append('None')

    # Get population density
    density = re.findall('\d+\.?\d+',
                        country_soup.find(lambda tag: tag.name == 'p' and 'population density in ' in tag.text).text.replace(',', ''))[0:2]
    populations.append(density[1])
    # Populations = [percentage urban, total population, population density per sqkm]

    # Append data as a list to demographics_data
    demographics_data.append([
        country_link.text.strip(),  # Country name
        *expectancies,              # Life expectancy (Both, Female, Male)
        *populations               # Urban population percentage, absolute, and density
    ])

# Create DataFrame from collected data
df_demographics = pd.DataFrame(
    demographics_data,
    columns=["Country", "LifeExpectancy_Both", "LifeExpectancy_Female", "LifeExpectancy_Male",
            "UrbanPopulation_Percentage", "UrbanPopulation_Absolute", "PopulationDensity"]
).apply(pd.to_numeric)


Scraping data for Afghanistan:   0%|          | 0/201 [00:01<?, ?it/s]


AttributeError: 'DataFrame' object has no attribute 'add_row'

In [136]:
df_demographics.to_csv('scraped_demographics.csv', index=False)

,Country,LifeExpectancy_Both,LifeExpectancy_Female,LifeExpectancy_Male,UrbanPopulation_Percentage,UrbanPopulation_Absolute,PopulationDensity
0,Afghanistan,66.5,68.1,64.9,26.7,11704638,67
1,Albania,80.0,81.7,78.1,70.3,1948831,101
2,Algeria,76.7,78.1,75.3,74.4,35291973,20
3,Angola,65.0,67.5,62.4,68.8,26847887,31
4,Antigua and Barbuda,77.9,80.6,74.9,28.5,26823,214


## Testing for country 0

In [28]:
(country_url := SCRAPE_ROOT + soup.find_all(attrs={'data-country': True})[0]['href'])

'https://www.worldometers.info/demographics/afghanistan-demographics/'

In [29]:
response = requests.get(country_url)

### Getting Life Expectancy

In [68]:
country_soup = BeautifulSoup(response.content, 'html.parser')
expectancies = map( lambda x: float(x.text.strip()), country_soup.find_all(attrs={'class': 'grid grid-col-1 lg:grid-cols-3 gap-4'})[0].find_all(
    attrs={'class': 'text-2xl font-bold mb-1.5'}))
list(expectancies)

[66.5, 68.1, 64.9]

### Getting Population

In [121]:
populations= re.findall('\d+\.?\d+',
                        country_soup.find(lambda tag: tag.name == 'p' and 'Currently' in tag.text).text.replace(',', ''))[0:2]
# Dealing with lack of absolute number
if len(populations)==1:
    populations.append('None')
populations

['26.7', '11704638']

### Getting Population Density

In [123]:
density= re.findall('\d+\.?\d+',
                        country_soup.find(lambda tag: tag.name == 'p' and 'population density in ' in tag.text).text.replace(',', ''))[0:2]
populations.append(density[1])
populations

['26.7', '11704638', 67.0]